In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

  Updating registry at `~/.juliapro/JuliaPro_v1.2.0-1/registries/JuliaPro`
  Updating git-repo `https://pkg.juliacomputing.com//registry/JuliaPro`
Fetching: [========================================>]  100.0 %.0 %Fetching: [=======>                                 ]  15.7 %>                        ]  38.0 %>                   ]  52.1 %>           ]  70.8 %Fetching: [=================================>       ]  81.7 %>  ]  94.5 %   Cloning git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`
  Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`ng: [===============>                         ]  36.1 %Fetching: [===================>                     ]  47.2 %Fetching: [====================>                    ]  49.4 %Fetching: [=====================>                   ]  51.1 %Fetching: [======================>                  ]  52.8 %Fetching: [======================>                  ]  54.5 %Fetching: [=======================>                 ]  56.

┌ Info: Precompiling JuliaAcademyData [18b7da76-0988-5e3b-acac-6290be3a708f]
└ @ Base loading.jl:1242


  Updating registry at `~/.juliapro/JuliaPro_v1.2.0-1/registries/JuliaPro`
  Updating git-repo `https://pkg.juliacomputing.com//registry/JuliaPro`
 Installed CUDAapi ──────────────── v0.6.3
 Installed Measures ─────────────── v0.3.0
 Installed Hexagons ─────────────── v0.2.0
 Installed Hwloc ────────────────── v1.0.0
 Installed NNlib ────────────────── v0.5.0
 Installed CSTParser ────────────── v0.5.2
 Installed Conda ────────────────── v1.2.0
 Installed OffsetArrays ─────────── v0.10.0
 Installed Rmath ────────────────── v0.5.0
 Installed Arpack ───────────────── v0.2.3
 Installed NLSolversBase ────────── v7.3.1
 Installed IterTools ────────────── v1.1.1
 Installed CategoricalArrays ────── v0.5.2
 Installed TimerOutputs ─────────── v0.5.0
 Installed DiffEqDiffTools ──────── v0.8.1
 Installed Juno ─────────────────── v0.7.0
 Installed KernelDensity ────────── v0.5.1
 Installed ForwardDiff ──────────── v0.10.3
 Installed Flux ─────────────────── v0.8.2
 Installed NaNMath ───────────────

┌ Error: Error building `Homebrew`: 
######################################################################### 100.0%
            -=O=-      #     #     #     #                                     
-=O#- #  #    #                                                                
-=O=-                  #     #     #     #                                     
│ [ Info: Downloading brew...
│ ERROR: LoadError: InitError: KeyError: key "10.15" not found
│ Stacktrace:
│  [1] getindex at ./dict.jl:477 [inlined]
│  [2] osx_version_string() at /Users/paoluzzi/.juliapro/JuliaPro_v1.2.0-1/packages/Homebrew/s09IX/src/API.jl:633
│  [3] has_bottle(::String) at /Users/paoluzzi/.juliapro/JuliaPro_v1.2.0-1/packages/Homebrew/s09IX/src/API.jl:654
│  [4] #translate_formula#71(::Bool, ::typeof(Homebrew.translate_formula), ::String) at /Users/paoluzzi/.juliapro/JuliaPro_v1.2.0-1/packages/Homebrew/s09IX/src/translation.jl:135
│  [5] #translate_formula at ./tuple.jl:0 [inlined]
│  [6] #translate_formula#78 at /

SystemError: SystemError: opening file "/Users/paoluzzi/.juliapro/JuliaPro_v1.2.0-1/packages/Hwloc/aHXIz/deps/build.log": Permission denied

# Distributed (or multi-core or multi-process) parallelism

Julia has a built-in standard library — Distributed — that allows you to
start and run multiple concurrent Julia processes. Imagine starting a slew
of Julia instances and then having an easy way to run code on each and every
one of them; that's what Distributed provides.

![](https://raw.githubusercontent.com/JuliaComputing/JuliaAcademyData.jl/master/courses/Parallel_Computing/images/Julia6x.png)

In [2]:
using Distributed
nprocs()

1

In [3]:
addprocs(4)
@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
nprocs()

5

In [4]:
myid()

1

Now we can easily communicate with the other nodes:

In [5]:
r = @spawnat 2 (myid(), rand())

Future(2, 1, 14, nothing)

In [6]:
fetch(r)

(2, 0.3577998139355627)

This works kinda like an `@async` task!

In [7]:
@time r = @spawnat 2 (sleep(1), rand())
@time fetch(r)

  0.000270 seconds (116 allocations: 6.698 KiB)
  1.157857 seconds (43.87 k allocations: 2.225 MiB, 0.51% gc time)


(nothing, 0.6245654714411526)

So we can repeat the same examples from tasks:

In [9]:
@time for i in 2:nprocs() # proc 1 is the controlling node
    @spawnat i sleep(1)
end

  0.000820 seconds (348 allocations: 20.109 KiB)


In [10]:
@time @sync for i in 2:nprocs()
    @spawnat i sleep(1)
end

  1.200191 seconds (43.91 k allocations: 2.204 MiB)


Except unlike tasks, we're executing the code on a separate process — which
can be performed on a different processor in parallel!

In [11]:
@everywhere function work(N)
    series = 1.0
    for i in 1:N
        series += (isodd(i) ? -1 : 1) / (i*2+1)
    end
    return 4*series
end

In [12]:
@time work(1_000_000_000)
@time @sync for i in workers()
    @spawnat i work(1_000_000_000)
end

  4.373346 seconds (31.00 k allocations: 1.705 MiB)
  4.839907 seconds (1.43 k allocations: 67.359 KiB)


Of course, this isn't very helpful. We're just performing exactly the same
calculation on every worker... and then completely ignoring the result! Let's
restructure our computation to be a bit more parallel friendly:

In [13]:
@everywhere function partial_pi(r)
    series = 0.0
    for i in r
        series += (isodd(i) ? -1 : 1) / (i*2+1)
    end
    return 4*series
end
a = partial_pi(0:999)
a, a-pi

(3.140592653839794, -0.000999999749998981)

In [15]:
b = partial_pi(1000:9999)
(a + b), (a+b) - pi

(3.1414926535900443, -9.999999974885654e-5)

So now we can distribute this computation across our many workers!

In [16]:
r = 0:10_000_000_000
futures = Array{Future}(undef, nworkers())
@time begin
    for (i, id) in enumerate(workers())
        batch = 0:length(r)÷nworkers()-1
        futures[i] = @spawnat id partial_pi(batch .+ (i-1)*(length(r)÷nworkers()))
    end
    p = sum(fetch.(futures))
end
p - pi

 11.799564 seconds (442.28 k allocations: 23.116 MiB, 0.05% gc time)


-1.0152234608540311e-10

In [23]:
p,0. + π

(3.1415926534882708, 3.141592653589793)

But that's rather annoying — needing to carefully divide up our workflow and
manually collect all our results and such.  There's an easier way:

In [24]:
@time p = @distributed (+) for r in [(0:9999) .+ offset for offset in 0:10000:r[end]-1]
    partial_pi(r)
end
p - pi

 12.188914 seconds (1.11 M allocations: 72.572 MiB, 0.89% gc time)


-9.940004375152967e-11

Why is this different from `@threads for` and `@simd for`? Why not just
`@distributed for`?  Why the `@distributed (+) for`?

## Data movement

Remember: Moving data is _expensive_!

| System Event                   | Actual Latency | Scaled Latency |
| ------------------------------ | -------------- | -------------- |
| One CPU cycle                  |     0.4 ns     |     1 s        |
| Level 1 cache access           |     0.9 ns     |     2 s        |
| Level 2 cache access           |     2.8 ns     |     7 s        |
| Level 3 cache access           |      28 ns     |     1 min      |
| Main memory access (DDR DIMM)  |    ~100 ns     |     4 min      |
| Intel Optane memory access     |     <10 μs     |     7 hrs      |
| NVMe SSD I/O                   |     ~25 μs     |    17 hrs      |
| SSD I/O                        |  50–150 μs     | 1.5–4 days     |
| Rotational disk I/O            |    1–10 ms     |   1–9 months   |
| Internet call: SF to NYC       |      65 ms     |     5 years    |
| Internet call: SF to Hong Kong |     141 ms     |    11 years    |

You really don't want to be taking a trip to the moon very frequently.
Communication between processes can indeed be as expensive as hitting a disk —
sometimes they're even implemented that way.

So that's why Julia has special support for reductions built in to the
`@distributed` macro: each worker can do its own (intermediate) reduction
before returning just one value to our master node.

But sometimes you need to see those intermediate values. If you have a
very expensive computation relative to the communication overhead, there are
several ways to do this. The easiest is `pmap`:

In [25]:
@time pmap(partial_pi, [(0:99999) .+ offset for offset in 0:100000:r[end]-1])

 20.836519 seconds (10.18 M allocations: 342.269 MiB, 0.66% gc time)


100000-element Array{Float64,1}:
 3.1415826535897198    
 4.999999999780955e-6  
 1.6666666666445866e-6 
 8.333333333280828e-7  
 4.999999999980979e-7  
 3.3333333333228886e-7 
 2.3809523809478275e-7 
 1.7857142857115218e-7 
 1.3888888888875312e-7 
 1.1111111111103951e-7 
 9.090909090911855e-8  
 7.575757575759043e-8  
 6.410256410254195e-8  
 ⋮                     
 1.0002300392212729e-15
 1.0002100347887365e-15
 1.0001900271250258e-15
 1.0001700234938206e-15
 1.0001500160110557e-15
 1.0001300152749827e-15
 1.000110006034459e-15 
 1.0000900041093138e-15
 1.0000700028304034e-15
 1.0000500022235774e-15
 1.000029998049535e-15 
 1.000010000079347e-15 

But if we have a large computation relative to the number of return values,
pmap is great and easy.

Increase the work on each worker by 100x and reduce the amount of communication by 100x:

In [26]:
@time pmap(partial_pi, [(0:9999999) .+ offset for offset in 0:10000000:r[end]-1])

 12.406665 seconds (170.51 k allocations: 6.801 MiB)


1000-element Array{Float64,1}:
 3.1415925535897915    
 4.9999999999967625e-8 
 1.6666666666683624e-8 
 8.333333333630988e-9  
 5.000000000014345e-9  
 3.333333333330306e-9  
 2.3809523814998224e-9 
 1.7857142857113054e-9 
 1.3888888888936273e-9 
 1.111111111107229e-9  
 9.09090909089179e-10  
 7.575757575755141e-10 
 6.410256410217499e-10 
 ⋮                     
 1.0234031840540388e-13
 1.021335702983439e-13 
 1.0192744799477636e-13
 1.0172194917187468e-13
 1.0151707113535652e-13
 1.013128113994666e-13 
 1.0110916758200228e-13
 1.0090613714424288e-13
 1.0070371752733102e-13
 1.0050190652654602e-13
 1.0030070151409018e-13
 1.0010010009995436e-13

There are other ways of doing this, though, too — we'll get to them in a minute.
But first, there's something else that I glossed over: the `@everywhere`s above.

## Code movement

Each node is _completely_ independent; it's like starting brand new, separate
Julia processes yourself. By default, `addprocs()` just launches the
appropriate number of workers for the current workstation that you're on, but
you can easily connect them to remote machines via SSH or even through cluster
managers.

Those `@everywhere`s above are very important! They run the given expression
on all workers to make sure the state between them is consistent. Without it,
you'll see errors like this:

In [27]:
hello() = "hello world"
r = @spawnat 2 hello()

Future(2, 1, 101070, nothing)

In [28]:
fetch(r)

RemoteException: On worker 2:
UndefVarError: #hello not defined
deserialize_datatype at /Applications/JuliaPro-1.2.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
handle_deserialize at /Applications/JuliaPro-1.2.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
deserialize at /Applications/JuliaPro-1.2.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
handle_deserialize at /Applications/JuliaPro-1.2.0-1.app/Contents/Resources/julia/Contents/Resources/julia/lib/julia/sys.dylib:?
deserialize_global_from_main at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:722
#5 at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Distributed/src/clusterserialize.jl:72 [inlined]
foreach at ./abstractarray.jl:1920
dese

Note that this applies to packages, too!

In [29]:
using Statistics # The Statistics stdlib defines mean
fetch(@spawnat 2 mean(rand(100_000)))

RemoteException: On worker 2:
UndefVarError: mean not defined
#27 at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Distributed/src/macros.jl:73
#109 at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:288
run_work_thunk at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:79
run_work_thunk at /Users/vagrant/worker/juliapro-release-osx1011-0_6/build/tmp_julia/Julia-1.2.app/Contents/Resources/julia/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:88
#102 at ./task.jl:268

In [30]:
@everywhere using Statistics
fetch(@spawnat 2 mean(rand(100_000)))

0.5011824602391834

# Other ways to structure and/or share data between processes

Unlike `@threads`, we no longer have access to the same memory. While this
does make expressing some algorithms a little more tricky, the "default"
is much safer! There isn't any shared state to begin with, so it's harder
to write an incorrect algorithm. It's also just harder to write some
algorithms in the first place.

So there are some special array types that can help bridge the gap between
processes and make writing parallel code a bit easier.

## The `SharedArray`

If all workers are on the same physical machine, while they cannot share
memory, they do all have shared access to the same hard drive(s)!

The `SharedArray` makes use of this fact, allowing concurrent accesses to the
same array — somewhat akin to threads default state.

This is the prefix definition from the "thinking in parallel" course:

```
using .Threads
function prefix_threads!(y, ⊕)
    l=length(y)
    k=ceil(Int, log2(l))
    for j=1:k
        @threads for i=2^j:2^j:min(l, 2^k)       #"reduce"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    for j=(k-1):-1:1
        @threads for i=3*2^(j-1):2^j:min(l, 2^k) #"expand"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    y
end
```

In [ ]:
using SharedArrays
function prefix!(y::SharedArray, ⊕)
    l=length(y)
    k=ceil(Int, log2(l))
    for j=1:k
        @distributed for i=2^j:2^j:min(l, 2^k)       #"reduce"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    for j=(k-1):-1:1
        @distributed for i=3*2^(j-1):2^j:min(l, 2^k) #"expand"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    y
end
data = rand(1_000_000);
A = SharedArray(data);

In [ ]:
prefix!(SharedArray(data), +) # compile
@time prefix!(A, +);

In [ ]:
A ≈ cumsum(data)

What went wrong?

In [ ]:
function prefix!(y::SharedArray, ⊕)
    l=length(y)
    k=ceil(Int, log2(l))
    for j=1:k
        @sync @distributed for i=2^j:2^j:min(l, 2^k)       #"reduce"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    for j=(k-1):-1:1
        @sync @distributed for i=3*2^(j-1):2^j:min(l, 2^k) #"expand"
            @inbounds y[i] = y[i-2^(j-1)] ⊕ y[i]
        end
    end
    y
end
A = SharedArray(data)
@time prefix!(A, +)

In [ ]:
A ≈ cumsum(data)

## DistributedArrays

We can, though, turn the problem on its head and allow the _data_ itself
to determine how the problem gets split up. This can save us tons of indexing
headaches.

In [ ]:
@everywhere using Distributed
@everywhere using DistributedArrays
A = DArray(I->fill(myid(), length.(I)), (24, 24))

The first argument takes a function that transforms the given set of indices
to the _local portion_ of the distributed array.

In [ ]:
A = DArray((24,24)) do I
    @show I
    fill(myid(), length.(I))
end

Notice that none of the array actually lives on processor 1, but we can still
display the contents — when we do we're requesting all workers give us their
current data! While we've only talked about master-worker communcation so far,
workers can communicate directly amongst themselves, too (by default).

In [ ]:
@everywhere using BenchmarkTools
fetch(@spawnat 2 @benchmark $A[1,1])

In [ ]:
fetch(@spawnat 2 @benchmark $A[end,end])

So it's fastest to work on a `DArray`'s "local" portion, but it's _possible_
to grab other data if need be. This is perfect for any sort of tiled operation
that works on neighboring values (like image filtering/convolution). Or Conway's
game of life!

In [ ]:
function life_step(d::DArray)
    DArray(size(d),procs(d)) do I
        # Compute the indices of the outside edge (that will come from other processors)
        top   = mod1(first(I[1])-1,size(d,1))
        bot   = mod1( last(I[1])+1,size(d,1))
        left  = mod1(first(I[2])-1,size(d,2))
        right = mod1( last(I[2])+1,size(d,2))
        # Create a new, temporary array that holds the local part + outside edge
        old = Array{Bool}(undef, length(I[1])+2, length(I[2])+2)
        # These accesses will pull data from other processors
        old[1      , 1      ] = d[top , left]
        old[2:end-1, 1      ] = d[I[1], left]   # left side (and corners)
        old[end    , 1      ] = d[bot , left]
        old[1      , end    ] = d[top , right]
        old[2:end-1, end    ] = d[I[1], right]  # right side (and corners)
        old[end    , end    ] = d[bot , right]
        old[1      , 2:end-1] = d[top , I[2]]   # top
        old[end    , 2:end-1] = d[bot , I[2]]   # bottom
        # But this big one is all local!
        old[2:end-1, 2:end-1] = d[I[1], I[2]]   # middle

        life_rule(old) # Compute the new segment!
    end
end
@everywhere function life_rule(old)
    # Now this part — the computational part — is entirely local and on Arrays!
    m, n = size(old)
    new = similar(old, m-2, n-2)
    for j = 2:n-1
        @inbounds for i = 2:m-1
            nc = (+)(old[i-1,j-1], old[i-1,j], old[i-1,j+1],
                     old[i  ,j-1],             old[i  ,j+1],
                     old[i+1,j-1], old[i+1,j], old[i+1,j+1])
            new[i-1,j-1] = (nc == 3 || nc == 2 && old[i,j])
        end
    end
    new
end

In [ ]:
A = DArray(I->rand(Bool, length.(I)), (20,20))
using Colors
Gray.(A)

In [ ]:
B = copy(A)

In [ ]:
B = Gray.(life_step(B))

## Clusters and more ways to distribute

You can easily connect to completely separate machines with SSH access built in!
But there are many other ways to connect to clusters:

* [JuliaRun](https://juliacomputing.com/products/juliarun)
* [Kubernetes](https://juliacomputing.com/blog/2018/12/15/kuber.html)
* [MPI](https://github.com/JuliaParallel/MPI.jl)
* [Cluster job queues with ClusterManagers](https://github.com/JuliaParallel/ClusterManagers.jl)
* [Hadoop](https://github.com/JuliaParallel/Elly.jl)
* [Spark](https://github.com/dfdx/Spark.jl)

# Multi-process parallelism is the heavy-duty workhorse in Julia

It can tackle very large problems and distribute across a very large number
of workers. Key things to remember

* Each worker is a completely independent Julia process
    * Data must move to them
    * Code must move to them
* Structure your algorithms and use a distributed mechanism that fits with the
  time and memory parameters of your problem
    * `@distributed` can be good for reductions and even relatively fast inner loops with limited (or no) explicit data transfer
    * `pmap` is great for very expensive inner loops that return a value
    * `SharedArray`s can be an easier drop-in replacement for threading-like behaviors (on a single machine)
    * `DistributedArray`s can turn the problem on its head and let the data do the work splitting!